# 2021/03/11 (Th) Study Session Questions

# 1. Reverse Sentence

## Problem

Your team is scrambling to decipher a recent message, worried it's a plot to break into a major European National Cake Vault. The message has been mostly deciphered, but all the words are backward! Your colleagues have handed off the last step to you.

> Write a function reverse_words() that takes a message as a list of characters and reverses the order of the words in place.

Why a list of characters instead of a string?
The goal of this question is to practice manipulating strings in place. Since we're modifying the message, we need a mutable ↴ type like a list, instead of Python 3.6's immutable strings.
For example:
```python
message = [ 'c', 'a', 'k', 'e', ' ',
            'p', 'o', 'u', 'n', 'd', ' ',
            's', 't', 'e', 'a', 'l' ]

reverse_words(message)

# Prints: 'steal pound cake'
print(''.join(message))
```

## Observation
1. words are delimited by whitespace characters
2. If we take no constraint approach, the most memory expensive operation is storing the entire word before placing it in the right part of the sentence.

## Approach

With no contraints, I would do the following.
```python
def reverseWord(word: [str]) -> str:
    return ''.join(word).split()[::-1]
```

lovely python magic that turns `[str]` to `str` then split it on white space char, then slice it backwards. Take O(N) Time O(N) space.

The tricky part about this is doing it `in-place` which means no additional memory that grows with size of the input - not necessarily no auxiliary variable. We've observed that storing an entire word to place in the right part of the sentence does not work. To avoid storing a whole word, we need a way to keep the word data without actually holding the word. This can be done by holding on to the start and end indices of a word. later we can access the word by giving it `message[word_start:word_end]` 

Then comes swapping the word into the right place. Best way to swap the location of structured date in-place is by buffer and replace three way swap that follows:
1. place elemA in the buffer
2. place elemB in position of elemA
3. place the buffer into elemB?
4. do this until the swap termination condition is met.

Based on this and the no constraint approach, my intuition is to:
1. reverse the entire array
2. have two ptrs, start/end of a word
3. move end until end hits an whitespace character
4. once it encounters a whitespace character, reverse the subarray that contains the word
5. move start to the start of a new word
6. repeat this until you are done with the reversed string

## Code

In [1]:
# Utility function for reversing a "word" in sentence -> really just reversing a string
def reverseString(word: [str]) -> [str]:
    buffer, start, end = '', 0, len(word)-1
    
    while start < end:
        buffer = word[start]
        word[start] = word[end]
        word[end] = buffer
        start += 1
        end -= 1
    return word

# Main function
def reverseWord(message: [str]) -> str:
    # 1. reverse the entire array
    message = reverseString(message)
    # 2. have two ptrs, start/end of a word
    start, end, messageSize = 0, 0, len(message)
    # 6. repeat this until you are done with the reversed string
    while end < messageSize:
        # 4. once it encounters a whitespace character, reverse the subarray that contains the word
        if message[end] == ' ':
            message[start:end] = reverseString(message[start:end])
            # 5. move start to the start of a new word
            start = end + 1
        # 3. move end until end hits an whitespace character
        end += 1
    # Because the last char of the array is not ' ', last word has not been fliped. Manually do this outside to loop.
    message[start:end] = reverseString(message[start:end])
    return ''.join(message)

## Test

In [2]:
message  = [ 'c', 'a', 'k', 'e', ' ',
            'p', 'o', 'u', 'n', 'd', ' ',
            's', 't', 'e', 'a', 'l' ]
expected = "steal pound cake"
actual   = reverseWord(message)
print("actual  : " + actual)
print(actual == expected)

actual  : steal pound cake
True


## Complexity

### Time
`O(n)` I am going through all elements once in the main while loop, then n/2 times when I am swapping the word. The dominant term here is the cost of the main while loop that loops through the entire message array, thus the time complexity is `O(n)`

*ignoring the cost for joining the array, because it takes O(n) and we already take it as a fixed cost*

### Space
`O(1)` We need 4 ptrs total. 2 to reverse a full word, and 2 to keep track of the word's position in the message. The implementation above seeks to reduce constant time operation of calculating len(message) by caching its value because this calculation happens N times. As the size of memory required for reverseWord() does not grow with the size of N, this is constant space `O(1)` operation.

# 2.Merge Array Sorted

## Problem
Each order is represented by an "order id" (an integer).
We have our lists of orders sorted numerically already, in lists. Write a function to merge our lists of orders into one sorted list.
For example:
```python
my_list     = [3, 4, 6, 10, 11, 15]
alices_list = [1, 5, 8, 12, 14, 19]

# Prints [1, 3, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
print(merge_lists(my_list, alices_list))
```

## Observation
1. Two lists contain only positive integers
2. Two lists are sorted in their own order.
3. Two lists are not necessarily of the same length
4. I need to visit every element in both lists at least once to merge them completely

## Approach

### Brute Force: Merge and Sort
I can merge two lists first, then sort the merged list. Assuming we use quicksort, the complexity of this operation becomes `O(n) + O(m) + O((n+m)log(n+m))`, and the overall complexity will be capped by `O((n+m)log(n+m))`. Space complexity is `O(n)` because we will need to increase the size of the larger array by the size of the smaller array in order to append the two.

1. merge the two lists
2. sort the merged lists

As we can assume that the total number of elements to go through would be n+m, we can reasonably assume that this will be `O((2n)log(2n))` which will be `O(nlogn)`.

The bottle neck here is simply the complexity of the sort. How do we avoid sorting the merged list?

### Two Pointer Comparison

The least time complexity has to be `O(n+m)`, where n and m are respective length of the lists. The length of two lists are not guaranteed to be the same.
As a human, I would do the following

1. Check the first elements of one array to another
2. Place the smaller element in a new, merged array
3. For the array where you took the element from, move down one number. 
4. Make the same comparison to the new element from array 1 and the current element form array 2
4. Do this until one of the array is fully traversed
5. Append what remains of the array with leftovers to the merged array list

## Code

In [3]:
def mergeThenSort(arr1: [int], arr2:[int]) -> [int]:
    merged = arr1+arr2
    merged.sort()
    return merged

In [4]:
def mergeTwoPtrCmp(arr1: [int], arr2: [int]) -> [int]:
        if not arr1 or not arr2: 
            return arr1+arr2

        ptr1, ptr2, merged, len1, len2 = 0, 0, [], len(arr1), len(arr2)
        # 5. Do this until one of the array is fully traversed
        while ptr1 < len1 and ptr2 < len2:
            # 1. Check the first element of one array to another
            if arr1[ptr1] <= arr2[ptr2]:
                # 2. Place the smaller element in a new, merged array
                merged.append(arr1[ptr1])
                # 3. For the array where you took the element from, move down one number. 
                ptr1 += 1
            else:
                # 2. Place the smaller element in a new, merged array
                merged.append(arr2[ptr2])
                # 3. For the array where you took the element from, move down one number. 
                ptr2 += 1
        # 6. Append what remains of the array with leftovers to the merged array list
        if ptr1 < len1:
            merged = merged + arr1[ptr1:]
        else:
            merged = merged + arr2[ptr2:]
        
        return merged

## Test

In [5]:
my_list     = [3, 4, 6, 10, 11, 15]
alices_list = [1, 5, 8, 12, 14, 19]
expected    = [1, 3, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
actual      = mergeTwoPtrCmp(my_list,alices_list)
print("actual: {}".format(actual))
print(actual == expected)

actual: [1, 3, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
True


In [6]:
my_list     = [3, 4, 6, 10, 11, 15]
alices_list = [1, 5, 8, 12, 14, 19]
expected    = [1, 3, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
actual      = mergeThenSort(my_list, alices_list)
print("actual: {}".format(actual))
print(actual == expected)

actual: [1, 3, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
True


## Complexity

### Time
- Merge then Sort    : `O((n+m) log (n+m))` because we are sorting a list size of (n+m)
- Two Ptr Comparison : `O(n+m)` because we are going over all elements of size n and m exactly once

### Space
- Merge then Sort    : `O(log(n+m))` because quicksort requires `O(log(n+m))` on the call stack
- Two Ptr Comparison : `O(n+m)` because Additional space to copy over the full two arrays

## Follow Up

I can reduce the memory usage to `O(m)` of the smaller array. As `len(arr)` has time complexity of `O(1)` I can take the length of two arrays to get the size of the smaller of the two array. Then, append an empty array of size m to the end of larger array. To avoid overwriting the data, I can have three ptrs, one at the last value of larger array `ptr1` and one at the end of the smaller array `ptr2`. Lastly, have a pointer to the end of the large, concatted array `ptrWrite`. Then, make the comparison from the two pointers pointing at actual values and place the larger element at the location of `ptrWrite`. This is essentially sorting in a decending order, while traversing backwards. This is to avoid holding values of both arrays separately. If we sort from the start, then we are comparing the values that CANNOT yet be overwritten. By writing from the back of an empty list, we have the buffer to be sure that if any values are overwritten we know those values are already tracked at the later half of the appended list.

## Code

In [7]:
def bufferOverwriteMerge(arr1: [int], arr2: [int]) -> [int]:
    # Null case handling
    if not arr1 or not arr2:
        return arr1+arr2
    
    # Get the initial length of both lists
    len1, len2 = len(arr1)-1, len(arr2)-1
    
    # append array of size m to the larger list
    if len2 <= len1:
        arr1 = arr1 + [0 for x in arr2]
    else:
        arr2 = arr2 + [0 for x in arr1]
    
    # assign pointers - make sure to add 1 to ptrWrite because we are taking n-1 for both lengths for index 0
    ptr1, ptr2, ptrWrite = len1, len2, len1+len2+1
    
    # AND loop condition to exhaust at least one of the list's value
    while ptr1 >= 0 and ptr2 >= 0:
        # start by writing larger value at the end of the appended array and decrement counter of the list
        if arr1[ptr1] > arr2[ptr2]:
            arr1[ptrWrite] = arr1[ptr1]
            ptr1 -= 1
        else:
            arr1[ptrWrite] = arr2[ptr2]
            ptr2 -= 1
        ptrWrite -= 1
    
    # Once the loop terminates check which list remains to assign to the proper indices of the appended array
    if ptr1 <= ptr2:
        while ptr2 >=0:
            arr1[ptrWrite] = arr2[ptr2]
            ptr2 -= 1
            ptrWrite -= 1
    else:
        while ptr1 >= 0:
            arr1[ptrWrite] = arr1[ptr1]
            ptr1 -= 1
            ptrWrite -=1
    return arr1

## Test

In [8]:
my_list     = [3, 4, 6, 10, 11, 15]
alices_list = [1, 5, 8, 12, 14, 19]
expected    = [1, 3, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
actual      = bufferOverwriteMerge(my_list,alices_list)
print("actual: {}".format(actual))
print(actual == expected)

actual: [1, 3, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
True


## Complexity

### Time
We are still going through all elements in both arrays, so `O(n+m)`

### Space
We are saving the space for the larger array because we can secure the values and overwrite the size of the larger array, so `O(m)`